<a href="https://colab.research.google.com/github/khushidubeyokok/BERTopic/blob/main/corelate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets
!pip -q install bertopic
!pip -q install transformers
!pip -q install umap-learn hdbscan
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("neuralwork/arxiver")

# Sample view of dataset
print("Sample Titles and Abstracts:")
for i in range(3):
    print(f"Title {i+1}: {dataset['train'][i]['title']}")
    print(f"Abstract {i+1}: {dataset['train'][i]['abstract']}\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63357 [00:00<?, ? examples/s]

Sample Titles and Abstracts:
Title 1: Image Completion via Dual-path Cooperative Filtering
Abstract 1: Given the recent advances with image-generating algorithms, deep image
completion methods have made significant progress. However, state-of-art
methods typically provide poor cross-scene generalization, and generated masked
areas often contain blurry artifacts. Predictive filtering is a method for
restoring images, which predicts the most effective kernels based on the input
scene. Motivated by this approach, we address image completion as a filtering
problem. Deep feature-level semantic filtering is introduced to fill in missing
information, while preserving local structure and generating visually realistic
content. In particular, a Dual-path Cooperative Filtering (DCF) model is
proposed, where one path predicts dynamic kernels, and the other path extracts
multi-level features by using Fast Fourier Convolution to yield semantically
coherent reconstructions. Experiments on three chall

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Download necessary resources
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase, remove numbers and special characters
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\W+', ' ', text)
    # Remove stopwords and lemmatize
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing
processed_titles = [preprocess_text(title) for title in dataset['train']['title']]
processed_abstracts = [preprocess_text(abstract) for abstract in dataset['train']['abstract']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic

# BERTopic for Titles
topic_model_titles = BERTopic(
    embedding_model="all-MiniLM-L6-v2",
    umap_model=UMAP(n_neighbors=10, n_components=5, min_dist=0.1, metric='cosine'),
    hdbscan_model=HDBSCAN(min_cluster_size=60, min_samples=15, metric='euclidean', cluster_selection_method='eom'),
    top_n_words=10
)
topics_titles,probs_titles = topic_model_titles.fit_transform(processed_titles)

# BERTopic for Abstracts
topic_model_abstracts = BERTopic(
    embedding_model="all-MiniLM-L6-v2",
    umap_model=UMAP(n_neighbors=10, n_components=5, min_dist=0.1, metric='cosine'),
    hdbscan_model=HDBSCAN(min_cluster_size=60, min_samples=15, metric='euclidean', cluster_selection_method='eom'),
    top_n_words=10
)
topics_abstracts, probs_abstracts = topic_model_abstracts.fit_transform(processed_abstracts)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
title_topic_words = {i: topic_model_titles.get_topic(i) for i in range(len(set(topics_titles)))}
abstract_topic_words = {i: topic_model_abstracts.get_topic(i) for i in range(len(set(topics_abstracts)))}

In [8]:
# Get topic words for titles and abstracts
title_topic_words = {i: topic_model_titles.get_topic(i) for i in set(topics_titles)}
abstract_topic_words = {i: topic_model_abstracts.get_topic(i) for i in set(topics_abstracts)}

# Corrected function to handle topic words
def get_topic_vector(topic_words):
    # Extract only the words from tuples
    words_only = [word for word, _ in topic_words]
    return embedding_model.encode(' '.join(words_only))

# Get vectors for both title and abstract topics
title_vectors = {i: get_topic_vector(words) for i, words in title_topic_words.items()}
abstract_vectors = {i: get_topic_vector(words) for i, words in abstract_topic_words.items()}

# Compute cosine similarity between each pair of title and abstract topics
similarity_matrix = cosine_similarity(
    list(title_vectors.values()),
    list(abstract_vectors.values())
)


In [11]:
import pandas as pd

# Create a DataFrame to hold the results
data = []

for i in range(len(dataset['train'])):
    title = dataset['train'][i]['title']
    abstract = dataset['train'][i]['abstract']
    title_topic = topics_titles[i]
    abstract_topic = topics_abstracts[i]

    title_words = title_topic_words.get(title_topic, [])
    abstract_words = abstract_topic_words.get(abstract_topic, [])

    similarity_score = similarity_matrix[title_topic, abstract_topic]

    data.append({
        "Title": title,
        "Abstract": abstract,
        "Title Topic": title_topic,
        "Abstract Topic": abstract_topic,
        "Title Topic Words": title_words,
        "Abstract Topic Words": abstract_words,
        "Similarity": similarity_score
    })

df_similarity = pd.DataFrame(data)

# Show the DataFrame
print(df_similarity.head())
display(df_similarity)

                                               Title  \
0  Image Completion via Dual-path Cooperative Fil...   
1  High Sensitivity Beamformed Observations of th...   
2  Maybe, Maybe Not: A Survey on Uncertainty in V...   
3  Enhancing GAN-Based Vocoders with Contrastive ...   
4      Nonvolatile Magneto-Thermal Switching in MgB2   

                                            Abstract  Title Topic  \
0  Given the recent advances with image-generatin...           -1   
1  We analyzed four epochs of beamformed EVN data...            0   
2  Understanding and evaluating uncertainty play ...           79   
3  Vocoder models have recently achieved substant...          100   
4  Ongoing research explores thermal switching ma...            5   

   Abstract Topic                                  Title Topic Words  \
0              -1  [(model, 0.006355591375377003), (learning, 0.0...   
1               0  [(galaxy, 0.036399673816071294), (star, 0.0298...   
2              -1  [(visualizati

,Title,Abstract,Title Topic,Abstract Topic,Title Topic Words,Abstract Topic Words,Similarity
0,Image Completion via Dual-path Cooperative Fil...,Given the recent advances with image-generatin...,-1,-1,"[(model, 0.006355591375377003), (learning, 0.0...","[(model, 0.0076968741192351385), (data, 0.0066...",0.634382
1,High Sensitivity Beamformed Observations of th...,We analyzed four epochs of beamformed EVN data...,0,0,"[(galaxy, 0.036399673816071294), (star, 0.0298...","[(mass, 0.014331827276128807), (star, 0.013226...",0.484661
2,"Maybe, Maybe Not: A Survey on Uncertainty in V...",Understanding and evaluating uncertainty play ...,79,-1,"[(visualization, 0.15440414766423816), (interp...","[(model, 0.0076968741192351385), (data, 0.0066...",0.017764
3,Enhancing GAN-Based Vocoders with Contrastive ...,Vocoder models have recently achieved substant...,100,4,"[(gan, 0.20412203640993695), (inpainting, 0.10...","[(speech, 0.055290298997391144), (audio, 0.039...",0.018994
4,Nonvolatile Magneto-Thermal Switching in MgB2,Ongoing research explores thermal switching ma...,5,1,"[(spin, 0.05654174389575797), (hall, 0.0517168...","[(quantum, 0.027634509865103293), (state, 0.01...",0.381808
...,...,...,...,...,...,...,...
63352,Almost paratopological groups,A class of almost paratopological groups is in...,-1,96,"[(model, 0.006355591375377003), (learning, 0.0...","[(group, 0.11509787674206674), (subgroup, 0.04...",-0.018942
63353,ACL-Fig: A Dataset for Scientific Figure Class...,Most existing large-scale academic search engi...,121,-1,"[(captioning, 0.38176362140899733), (caption, ...","[(model, 0.0076968741192351385), (data, 0.0066...",0.202521
63354,Polynomial Time and Private Learning of Unboun...,We study the problem of privately estimating t...,23,62,"[(privacy, 0.2047630878009487), (private, 0.13...","[(privacy, 0.12141945050243498), (dp, 0.056217...",0.811114
63355,Improving Knot Prediction in Wood Logs with Lo...,The quality of a wood log in the wood industry...,126,-1,"[(knot, 0.4976789965743925), (link, 0.10898286...","[(model, 0.0076968741192351385), (data, 0.0066...",-0.064145
